In [23]:
import os
from tqdm import tqdm

from src.Utility import loadDir
from src.Analysis import Analysis
from src.Database import Database

In [24]:
class Config:
    tracks = 100
    segments = 10
    seconds = 3
    rate = 22050
    data_dir = 'data'
    lib_dir = 'D:\\Musik\\Genres\\'
    genres = [e for e in os.listdir(lib_dir) if e not in ['Trash', 'Other', 'JumpUp', 
               'Hiphop','Disco', 'Rock', 'Blues', 'Classical', 'Country', 'Reggae', 'Jazz', 'Metal', 'Pop',
               'DnB', 
            #    'Crossbread', 'Neurofunk', 'Sambass', 'Jungle', 'Drumstep', 'Deep', 'Liquid', 'Minimal', 'Dancefloor',
            #    'JumpUp', 
               'Techno', 'Goa', 'PsyTrance', 
               'Hardtekk', 'Riddim', 'Breakcore', 'Dubstep']]
    sub_dir = os.path.join(data_dir, f'{tracks}x{segments}x{seconds}-{len(genres)}')
    seed = 42
config = Config()
print(config.genres)

['Shriek', 'UKay']


In [25]:
def isAcceptedFormat(file: str) -> bool:
    return os.path.splitext(file)[-1] in ['.wav', '.mp3']

In [26]:
db = Database(config.sub_dir)
for index_dir, dir in enumerate(config.genres):
    files, genre = loadDir(os.path.join(config.lib_dir, dir))
    db.update(genre)
    if(len(files) == 0):
        print(f' -> {index_dir + 1}.\t{genre}: empty.')
        continue

    if(len(files) < config.tracks):
        print( f' -> {index_dir + 1}.\t{genre}: missing {config.tracks - len(files)}')
        continue

    if (db.count('label', genre) / config.segments >= config.tracks):
        print(f' -> {index_dir + 1}.\t{genre}: satisfied with {db.count("label", genre)} rows.')
        continue

    __bar = tqdm(total=(config.tracks) * config.segments, 
                 desc=f' -> {index_dir + 1}.\t{genre}:',
                 bar_format="{desc}\t{bar:20} {n_fmt}/{total_fmt} [{elapsed} -> {remaining}]",
                 initial=db.count('label', genre))

    for file_path in files:
        if(db.count('label', genre) / config.segments >= config.tracks):
            break

        # if not isAcceptedFormat(file_path):
        #     continue

        if(db.alreadyExsist(file_path)):
            continue 

        try:
            alyis = Analysis(audio_path=file_path, options=config, _bar = __bar)
            db.push(alyis.calculate(), file_path)
        except Exception:
            continue


        db.save()
    __bar.close()

 -> 1.	Shriek: satisfied with 1000 rows.
 -> 2.	UKay: satisfied with 1000 rows.
